# Multivariate Time Series Forecasting with Transformers 
Dataset: http://bit.ly/covid19-poland

In [124]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import networkx
import seaborn
import sklearn
from tabulate import tabulate

## Load dataset

In [125]:
df = pd.read_csv('DataSets/COVID-19_w_Polsce-Wzrost.csv', sep=';', usecols=["Data", "Nowe przypadki", "Zmiana liczby aktywnych przypadków"])
print(df.head())
print(df.tail())
print(f"Number of records: {len(df)}")
print("Labels types:")
print(f"{df.dtypes}")

# table = df.values.tolist()
# headers = df.columns.tolist()
# print(tabulate(table, headers=headers, tablefmt="pretty"))

# styled_df = df.style.format('{:^10}').set_properties(**{'border': '1px solid black', 'text-align': 'center'}).set_table_styles([{'selector': 'th', 'props': [('border', '1px solid black')]}])
# styled_df

   Data Nowe przypadki Zmiana liczby aktywnych przypadków
0  3.03            + 0                                + 0
1  4.03            + 1                                + 1
2  5.03            + 0                                + 0
3  6.03            + 4                                + 4
4  7.03            + 1                                + 1
      Data Nowe przypadki Zmiana liczby aktywnych przypadków
738  11.03        + 11637                              2 917
739  12.03            NaN                          5 169 557
740    NaN            NaN                                NaN
741    NaN            NaN                                NaN
742    NaN            NaN                                NaN
Number of records: 743
Labels types:
Data                                  float64
Nowe przypadki                         object
Zmiana liczby aktywnych przypadków     object
dtype: object


## Data preprocessing

### Remove NaNs

In [126]:
df = df.dropna()
# for index, row in df.iterrows():
#     if pd.isna(row['Data']):
#         df.drop(index, inplace=True)
print(f"Number of records after dropping all NaNs: {len(df)}")
df

Number of records after dropping all NaNs: 739


,Data,Nowe przypadki,Zmiana liczby aktywnych przypadków
0,3.03,+ 0,+ 0
1,4.03,+ 1,+ 1
2,5.03,+ 0,+ 0
3,6.03,+ 4,+ 4
4,7.03,+ 1,+ 1
...,...,...,...
734,7.03,+ 5585,-6 470
735,8.03,+ 13152,2 945
736,9.03,+ 14415,4 355
737,10.03,+ 13438,4 794


### Convert types

In [117]:
def is_string(value):
    return isinstance(value, str)
result = df.applymap(is_string)
print(result)

      Data  Nowe przypadki  Zmiana liczby aktywnych przypadków
0    False            True                                True
1    False            True                                True
2    False            True                                True
3    False            True                                True
4    False            True                                True
..     ...             ...                                 ...
734  False            True                                True
735  False            True                                True
736  False            True                                True
737  False            True                                True
738  False            True                                True

[739 rows x 3 columns]


In [118]:
df

,Data,Nowe przypadki,Zmiana liczby aktywnych przypadków
0,3.03,+ 0,+ 0
1,4.03,+ 1,+ 1
2,5.03,+ 0,+ 0
3,6.03,+ 4,+ 4
4,7.03,+ 1,+ 1
...,...,...,...
734,7.03,+ 5585,-6 470
735,8.03,+ 13152,2 945
736,9.03,+ 14415,4 355
737,10.03,+ 13438,4 794


In [119]:
print(df.dtypes)
typ_komorki = df.at[0, 'Nowe przypadki']

print("Typ komórki:", type(typ_komorki))

try:
    wartosc_int = int(typ_komorki)
    print("Przekonwertowana wartość na int:", wartosc_int)
except ValueError:
    print("Nie można przekonwertować wartości na int, ponieważ nie jest to liczba całkowita.")

Data                                  float64
Nowe przypadki                         object
Zmiana liczby aktywnych przypadków     object
dtype: object
Typ komórki: <class 'str'>
Nie można przekonwertować wartości na int, ponieważ nie jest to liczba całkowita.


In [120]:

for idx, wartosc_str in enumerate(df['Nowe przypadki']):
    print(f"{idx} {type(wartosc_str)}")
    wartosc_str = wartosc_str.strip().replace(' ', '')
    czy_minus = wartosc_str.startswith('-')

    try:
        wartosc_int = int(wartosc_str)
        if czy_minus:
            wartosc_int *= -1

        df.at[idx, 'Nowe przypadki'] = wartosc_int
    except ValueError:
        pass

print(df)


0 <class 'str'>
1 <class 'str'>
2 <class 'str'>
3 <class 'str'>
4 <class 'str'>
5 <class 'str'>
6 <class 'str'>
7 <class 'str'>
8 <class 'str'>
9 <class 'str'>
10 <class 'str'>
11 <class 'str'>
12 <class 'str'>
13 <class 'str'>
14 <class 'str'>
15 <class 'str'>
16 <class 'str'>
17 <class 'str'>
18 <class 'str'>
19 <class 'str'>
20 <class 'str'>
21 <class 'str'>
22 <class 'str'>
23 <class 'str'>
24 <class 'str'>
25 <class 'str'>
26 <class 'str'>
27 <class 'str'>
28 <class 'str'>
29 <class 'str'>
30 <class 'str'>
31 <class 'str'>
32 <class 'str'>
33 <class 'str'>
34 <class 'str'>
35 <class 'str'>
36 <class 'str'>
37 <class 'str'>
38 <class 'str'>
39 <class 'str'>
40 <class 'str'>
41 <class 'str'>
42 <class 'str'>
43 <class 'str'>
44 <class 'str'>
45 <class 'str'>
46 <class 'str'>
47 <class 'str'>
48 <class 'str'>
49 <class 'str'>
50 <class 'str'>
51 <class 'str'>
52 <class 'str'>
53 <class 'str'>
54 <class 'str'>
55 <class 'str'>
56 <class 'str'>
57 <class 'str'>
58 <class 'str'>
59 <cla

In [121]:
a = "- 1"
b = "+ 2"

a = a.replace(' ', '')
b = b.replace(' ', '')


a = int(a)
b = int(b)

print(a+1)
print(b)

0
2


In [122]:
df

,Data,Nowe przypadki,Zmiana liczby aktywnych przypadków
0,3.03,0,+ 0
1,4.03,1,+ 1
2,5.03,0,+ 0
3,6.03,4,+ 4
4,7.03,1,+ 1
...,...,...,...
734,7.03,5585,-6 470
735,8.03,13152,2 945
736,9.03,14415,4 355
737,10.03,13438,4 794


# TODO: DOKOŃCZYĆ PREPROCESSING

In [ ]:
# TODO: DOKOŃCZYĆ PREPROCESSING

## Visualisation

In [ ]:
def visualise():
    pass

## Create Model

In [ ]:
class TransformerModel(nn.Module):
    def __init__(self, input_size, output_size, num_layers=2, hidden_size=64, num_heads=4, dropout=0.1):
        super(TransformerModel, self).__init__()
        encoder_layer = nn.TransformerEncoderLayer(d_model=input_size, nhead=num_heads, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, src):
        src = src.permute(1, 0, 2)  # wymagane wymiary [seq_len, batch, input_size]
        output = self.transformer_encoder(src)
        output = self.fc(output[-1, :, :])  # ostatni element z sekwencji
        return output

## Training

In [ ]:
data = df[['Data', 'Nowe przypadki', 'Zmiana liczby aktywnych przypadków']].copy()
data['Data'] = pd.to_datetime(data['Data'])
data.set_index('Data', inplace=True)

data = (data - data.mean()) / data.std()

train_data = data.iloc[:int(0.8 * len(data))]
test_data = data.iloc[int(0.8 * len(data)):]

X_train = torch.tensor(train_data.values, dtype=torch.float32)
X_test = torch.tensor(test_data.values, dtype=torch.float32)

In [128]:
def train_and_evaluate(model, epochs, criterion, optimizer):
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        output = model(X_train)
        loss = criterion(output, X_train)
        loss.backward()
        optimizer.step()
        if (epoch+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

    model.eval()
    with torch.no_grad():
        predicted = model(X_test)
        predicted = predicted.detach().numpy()

## Evaluation

In [ ]:
input_size = 0
output_size = 0
num_layers = 0
hidden_size = 0
num_heads = 0
dropout = 0

model = TransformerModel(input_size, output_size, num_layers, hidden_size, num_heads, dropout)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
epochs = 100

train_and_evaluate(model, epochs, criterion, optimizer)

## Conclusion

In [130]:
# TODO: Do testowania proszę użyć kilku podejść podziału danych na uczące- testujace:
# TODO: (https://hub.packtpub.com/cross-validation-strategies-for-time-series-forecasting-tutorial/)
# TODO: -K-Fold, Time SeriesSplit, Blocking Time Series Split
# TODO: Dla każdego wyniku stosujemy różne miary błędu (ewaluacji) MSE, MAE, R^2,  specyficzność czułość, index of agreement itp..

SyntaxError: invalid syntax (1513674348.py, line 1)